---

# 0. Imports

In [63]:
# Basic imports
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

# Functional imports
import json
import pyarrow
from pickle import dump
import gc

# Model imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import f_classif, SelectKBest

pd.set_option('display.max_columns', None)

---

# 1. Data Collection

In [64]:
#df_2018 = pd.read_parquet("../data/raw/Combined_Flights_2018.parquet", engine="pyarrow")
#df_2019 = pd.read_parquet("../data/raw/Combined_Flights_2019.parquet", engine="pyarrow")
#df_2020 = pd.read_parquet("../data/raw/Combined_Flights_2020.parquet", engine="pyarrow")
df_2021 = pd.read_parquet("../data/raw/Combined_Flights_2021.parquet", engine="pyarrow")
df_2022 = pd.read_parquet("../data/raw/Combined_Flights_2022.parquet", engine="pyarrow")


In [65]:
#data_frames = [df_2018, df_2019, df_2020, df_2021, df_2022]
data_frames = [df_2021, df_2022]
df = pd.concat(data_frames, ignore_index=True)

In [66]:
df.head(3)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2021-03-03,SkyWest Airlines Inc.,SGU,PHX,False,False,724,714.0,0.0,-10.0,818.0,0.0,49.0,79.0,64.0,262.0,2021,1,3,3,3,AA,AA_CODESHARE,19805,AA,3133,OO,20304,OO,N728SK,3133,14794,1479405,34794,"St. George, UT",UT,49,Utah,87,14107,1410702,30466,"Phoenix, AZ",AZ,4,Arizona,81,0.0,-1.0,0700-0759,10.0,724.0,813.0,5.0,843,-25.0,0.0,-2.0,0800-0859,2,0.0
1,2021-03-03,SkyWest Airlines Inc.,PHX,SGU,False,False,922,917.0,0.0,-5.0,1031.0,0.0,48.0,78.0,74.0,262.0,2021,1,3,3,3,AA,AA_CODESHARE,19805,AA,3134,OO,20304,OO,N752SK,3134,14107,1410702,30466,"Phoenix, AZ",AZ,4,Arizona,81,14794,1479405,34794,"St. George, UT",UT,49,Utah,87,0.0,-1.0,0900-0959,23.0,940.0,1028.0,3.0,1040,-9.0,0.0,-1.0,1000-1059,2,0.0
2,2021-03-03,SkyWest Airlines Inc.,MHT,ORD,False,False,1330,1321.0,0.0,-9.0,1501.0,0.0,129.0,180.0,160.0,843.0,2021,1,3,3,3,AA,AA_CODESHARE,19805,AA,3135,OO,20304,OO,N766SK,3135,13296,1329604,30721,"Manchester, NH",NH,33,New Hampshire,14,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,-1.0,1300-1359,15.0,1336.0,1445.0,16.0,1530,-29.0,0.0,-2.0,1500-1559,4,0.0


---

# 2 Exploration and Data cleaning

## Explicación de Columnas - Dataset de Vuelos

### Información del Vuelo
* **FlightDate**: Fecha del vuelo (año, mes y día)
* **Airline**: Código de aerolínea operadora
* **Origin**: Código IATA aeropuerto origen
* **Dest**: Código IATA aeropuerto destino

### Indicadores de Interrupción
* **Cancelled**: Vuelo cancelado (1=Sí, 0=No)
* **Diverted**: Vuelo desviado (1=Sí, 0=No)

### Horarios y Retrasos
#### Salida
* **CRSDepTime**: Hora programada salida
* **DepTime**: Hora real salida
* **DepDelayMinutes**: Minutos de retraso salida
* **DepDelay**: Retraso salida (negativo=anticipado)

#### Llegada
* **ArrTime**: Hora real llegada
* **ArrDelayMinutes**: Minutos retraso llegada 
* **ArrDelay**: Retraso llegada (negativo=anticipada)

### Duración
* **AirTime**: Minutos en aire
* **CRSElapsedTime**: Tiempo estimado total
* **ActualElapsedTime**: Tiempo real total

### Geografía y Tiempo
* **Distance**: Distancia en millas
* **Year**: Año del vuelo
* **Quarter**: Trimestre (1-4)
* **Month**: Mes (1-12)
* **DayofMonth**: Día del mes
* **DayOfWeek**: Día semana (1=Lunes, 7=Domingo)

### Información Aerolíneas
* **Marketing_Airline_Network**: Aerolínea vendedora
* **Operating_Airline**: Aerolínea operadora física
* **DOT_ID_Marketing_Airline**: ID DOT aerolínea marketing
* **IATA_Code_Marketing_Airline**: Código IATA aerolínea marketing
* **Flight_Number_Marketing_Airline**: Número vuelo marketing
* **DOT_ID_Operating_Airline**: ID DOT aerolínea operadora
* **IATA_Code_Operating_Airline**: Código IATA aerolínea operadora

### Aeronave
* **Tail_Number**: Matrícula única
* **Flight_Number_Operating_Airline**: Número vuelo operativo

### Aeropuerto Origen
* **OriginAirportID**: ID único
* **OriginAirportSeqID**: ID secuencia
* **OriginCityMarketID**: ID mercado ciudad
* **OriginCityName**: Nombre ciudad
* **OriginState**: Abreviación estado
* **OriginStateFips**: Código FIPS estado
* **OriginStateName**: Nombre estado
* **OriginWac**: Código WAC

### Aeropuerto Destino
* **DestAirportID**: ID único
* **DestAirportSeqID**: ID secuencia
* **DestCityMarketID**: ID mercado ciudad
* **DestCityName**: Nombre ciudad
* **DestState**: Abreviación estado
* **DestStateFips**: Código FIPS estado
* **DestStateName**: Nombre estado
* **DestWac**: Código WAC

### Indicadores Retraso Salida
* **DepDel15**: Retraso >15 min (1=Sí, 0=No)
* **DepartureDelayGroups**: Grupos de retraso
* **DepTimeBlk**: Bloque horario (ej: "0700-0759")

### Movimientos Aeronave
* **TaxiOut**: Minutos en pista antes despegue
* **WheelsOff**: Hora despegue
* **WheelsOn**: Hora aterrizaje
* **TaxiIn**: Minutos en pista post-aterrizaje

### Indicadores Retraso Llegada
* **CRSArrTime**: Hora programada llegada
* **ArrDel15**: Retraso >15 min (1=Sí, 0=No)
* **ArrivalDelayGroups**: Grupos de retraso
* **ArrTimeBlk**: Bloque horario llegada

### Otros Indicadores
* **DistanceGroup**: Grupos por distancia
* **DivAirportLandings**: Aterrizajes no programados

In [67]:
print('Our dataframe contains {} rows and it has {} features.'.format(len(df), df.shape[1]))

Our dataframe contains 10390189 rows and it has 61 features.


---

## 2.2 Eliminating Irrelevant Features and dealing with null values

Eliminaremos algunos features antes de continuar con nuestro EDA para ahorrar en recursos y tiempo

In [68]:
columnas_necesarias = [
   'DayOfWeek',
   'Month',
   'Quarter',
   'DayofMonth',
   'DepDelayMinutes',
   'DepTime',
   'CRSDepTime',
   'Distance',
   'Airline',
   'OriginStateName',
   'DestStateName',
   'OriginCityName',
   'DestCityName'
]

In [69]:
df = df[columnas_necesarias]
df.head(3)

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName
0,3,3,1,3,0.0,714.0,724,262.0,SkyWest Airlines Inc.,Utah,Arizona,"St. George, UT","Phoenix, AZ"
1,3,3,1,3,0.0,917.0,922,262.0,SkyWest Airlines Inc.,Arizona,Utah,"Phoenix, AZ","St. George, UT"
2,3,3,1,3,0.0,1321.0,1330,843.0,SkyWest Airlines Inc.,New Hampshire,Illinois,"Manchester, NH","Chicago, IL"


In [70]:
df.columns

Index(['DayOfWeek', 'Month', 'Quarter', 'DayofMonth', 'DepDelayMinutes',
       'DepTime', 'CRSDepTime', 'Distance', 'Airline', 'OriginStateName',
       'DestStateName', 'OriginCityName', 'DestCityName'],
      dtype='object')

In [71]:
df.shape

(10390189, 13)

In [72]:
'''def estratificar_datos(df, target_col='DepDelayMinutes', sample_size=1000000):
    """
    Realiza un muestreo estratificado manteniendo las proporciones de las variables.
    
    Parámetros:
    df: DataFrame original
    
    target_col: Nombre de la columna objetivo
    sample_size: Tamaño deseado de la muestra
    """
    
    # Crear bins para la variable objetivo (DepDelayMinutes)
    df['delay_category'] = pd.qcut(df[target_col], q=10, labels=False, duplicates='drop')
    
    # Crear una categoría combinada para la estratificación
    df['strata'] = df['delay_category'].astype(str) + '_' + \
                   df['Airline'].astype(str) + '_' + \
                   df['Quarter'].astype(str)
    
    # Calcular el tamaño proporcional para cada estrato
    proportions = df['strata'].value_counts(normalize=True)
    
    # Inicializar DataFrame vacío para la muestra
    stratified_sample = pd.DataFrame()
    
    # Realizar el muestreo estratificado
    for stratum in proportions.index:
        stratum_size = int(proportions[stratum] * sample_size)
        if stratum_size > 0:
            stratum_sample = df[df['strata'] == stratum].sample(
                n=min(stratum_size, len(df[df['strata'] == stratum])),
                random_state=42
            )
            stratified_sample = pd.concat([stratified_sample, stratum_sample])
    
    # Eliminar columnas auxiliares
    stratified_sample = stratified_sample.drop(['delay_category', 'strata'], axis=1)
    
    return stratified_sample

# Uso del código
# Asumiendo que tu DataFrame original se llama 'df'
# df = pd.read_csv('tu_archivo.csv')

# Crear muestra estratificada
muestra_estratificada = estratificar_datos(
    df,
    target_col='DepDelayMinutes',
    sample_size= 1450000  # Ajusta este número según tus necesidades
)

# Verificar las proporciones
def verificar_proporciones(df_original, df_muestra):
    """
    Compara las proporciones de las variables clave entre el DataFrame original y la muestra
    """
    variables_check = ['Airline', 'Quarter', 'DepDelayMinutes']
    
    for var in variables_check:
        if var == 'DepDelayMinutes':
            # Para variables numéricas, comparamos estadísticas descriptivas
            print(f"\nEstadísticas de {var}:")
            print("\nOriginal:")
            print(df_original[var].describe())
            print("\nMuestra:")
            print(df_muestra[var].describe())
        else:
            # Para variables categóricas, comparamos proporciones
            print(f"\nProporciones de {var}:")
            print("\nOriginal:")
            print(df_original[var].value_counts(normalize=True))
            print("\nMuestra:")
            print(df_muestra[var].value_counts(normalize=True))

# Verificar las proporciones
verificar_proporciones(df, muestra_estratificada)'''

'def estratificar_datos(df, target_col=\'DepDelayMinutes\', sample_size=1000000):\n    """\n    Realiza un muestreo estratificado manteniendo las proporciones de las variables.\n    \n    Parámetros:\n    df: DataFrame original\n    \n    target_col: Nombre de la columna objetivo\n    sample_size: Tamaño deseado de la muestra\n    """\n    \n    # Crear bins para la variable objetivo (DepDelayMinutes)\n    df[\'delay_category\'] = pd.qcut(df[target_col], q=10, labels=False, duplicates=\'drop\')\n    \n    # Crear una categoría combinada para la estratificación\n    df[\'strata\'] = df[\'delay_category\'].astype(str) + \'_\' +                    df[\'Airline\'].astype(str) + \'_\' +                    df[\'Quarter\'].astype(str)\n    \n    # Calcular el tamaño proporcional para cada estrato\n    proportions = df[\'strata\'].value_counts(normalize=True)\n    \n    # Inicializar DataFrame vacío para la muestra\n    stratified_sample = pd.DataFrame()\n    \n    # Realizar el muestreo estra

### Análisis de Variables Seleccionadas para Predicción de Retrasos en Vuelos

### Variables Temporales
- `Month`, `DayOfWeek`: Capturan patrones estacionales y semanales
- `CRSDepTime`: Hora programada, afecta probabilidad de retrasos
- `DepTime`: Hora real de salida, necesaria para validación

### Variables Operativas
- `Airline`: Diferentes aerolíneas tienen distintos índices de puntualidad
- `Distance`: Factor crítico que afecta complejidad operativa
- `DepDelayMinutes`: Variable objetivo para predicción

### Variables Geográficas
- `Origin`, `Dest`: Identificadores de aeropuertos
- `OriginAirportID`: Identificador único para análisis consistente
- `OriginStateName`, `DestStateName`: Factores climatológicos regionales
- `DestCityName`: Patrones específicos de destinos urbanos

### Variables Excluidas y Razones
- Variables redundantes (ej: `DepDelay`, similar a `DepDelayMinutes`)
- Datos post-evento (ej: `ArrTime`, `AirTime`)
- Información detallada no predictiva (ej: `TailNumber`)
- Variables derivadas calculables desde las seleccionadas

## 2.3 Analisis de nuestra dataframe

In [73]:
df.describe()

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance
count,1.039019e+07,1.039019e+07,1.039019e+07,1.039019e+07,1.016128e+07,1.016143e+07,1.039019e+07,1.039019e+07
mean,4.010493e+00,5.837063e+00,2.293315e+00,1.576069e+01,1.402861e+01,1.329798e+03,1.326164e+03,7.964749e+02
std,2.006577e+00,3.189623e+00,1.047474e+00,8.776266e+00,4.937651e+01,4.942498e+02,4.806621e+02,5.865190e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,3.100000e+01
25%,2.000000e+00,3.000000e+00,1.000000e+00,8.000000e+00,0.000000e+00,9.210000e+02,9.170000e+02,3.690000e+02
50%,4.000000e+00,6.000000e+00,2.000000e+00,1.600000e+01,0.000000e+00,1.322000e+03,1.319000e+03,6.460000e+02
75%,6.000000e+00,8.000000e+00,3.000000e+00,2.300000e+01,7.000000e+00,1.735000e+03,1.729000e+03,1.034000e+03
max,7.000000e+00,1.200000e+01,4.000000e+00,3.100000e+01,7.223000e+03,2.400000e+03,2.359000e+03,5.812000e+03


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10390189 entries, 0 to 10390188
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DayOfWeek        int64  
 1   Month            int64  
 2   Quarter          int64  
 3   DayofMonth       int64  
 4   DepDelayMinutes  float64
 5   DepTime          float64
 6   CRSDepTime       int64  
 7   Distance         float64
 8   Airline          object 
 9   OriginStateName  object 
 10  DestStateName    object 
 11  OriginCityName   object 
 12  DestCityName     object 
dtypes: float64(3), int64(5), object(5)
memory usage: 1.0+ GB


***Coclusion:***

Nuestro data frame contiene los siguientes tipos de datos:

- ``Float``     --> 3
- ``Integer``   --> 4
- ``String``    --> 7



## 2.4 Buscando duplicados, valores vacios y lidiando con ellos

In [75]:
df.duplicated().sum()

202

In [76]:
df = df.drop_duplicates()

In [77]:
df.duplicated().sum()

0

In [78]:
df.isna().sum()[df.isna().sum()>0]

DepDelayMinutes    228885
DepTime            228735
dtype: int64

In [79]:
df = df.dropna()

In [80]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

***Conclusion:***

De aproximadamente 29M de filas se encontrarons solo 573 filas duplicadas y 762978 de valores nulos

Sabiendo estos numeros lo que haremos seremos eliminarlos para que no afecte en el analisis de datos. Esta decision fue tomada considerando que al ser un porcentaje bajo de data no afectara al momento del analisis

---

# 3. Casting

## 3.1 Dividing our dataset into categorical, numerical and temporal 

In [81]:
# Understanding how many type of features we have
df.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [82]:
num = ["float64", "int64"]
cat = ['O']
temp = ['datetime64']

df_num = df.select_dtypes(num)
df_cat = df.select_dtypes(cat)
df_temp = df.select_dtypes(temp)

In [83]:
#Numerical Dataframe
df_num.head(3)

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance
0,3,3,1,3,0.0,714.0,724,262.0
1,3,3,1,3,0.0,917.0,922,262.0
2,3,3,1,3,0.0,1321.0,1330,843.0


In [84]:
#Categorical Dataframe
df_cat.head(3)

,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName
0,SkyWest Airlines Inc.,Utah,Arizona,"St. George, UT","Phoenix, AZ"
1,SkyWest Airlines Inc.,Arizona,Utah,"Phoenix, AZ","St. George, UT"
2,SkyWest Airlines Inc.,New Hampshire,Illinois,"Manchester, NH","Chicago, IL"


In [85]:
#Temporal_Dataframe
df_temp.head(3)

""
0
1
2


---

# 4. Univariated Analysis

## 4.1 Categorical Variable Analysis 

In [86]:
df_cat.head(3)

,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName
0,SkyWest Airlines Inc.,Utah,Arizona,"St. George, UT","Phoenix, AZ"
1,SkyWest Airlines Inc.,Arizona,Utah,"Phoenix, AZ","St. George, UT"
2,SkyWest Airlines Inc.,New Hampshire,Illinois,"Manchester, NH","Chicago, IL"


In [87]:
'''fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(10,20))
sns.set_palette("bright")

# Airline
airline_counts = df_cat["Airline"].value_counts()
airlines_filtered = airline_counts[airline_counts >= 50000]
sns.countplot(ax=ax[0], 
              data=df_cat[df_cat["Airline"].isin(airlines_filtered.index)],
              y="Airline", 
              order=airlines_filtered.index,
              hue="Airline")

# Origin State
origin_state_counts = df_cat["OriginStateName"].value_counts()
origin_states_filtered = origin_state_counts[origin_state_counts >= 50000]
sns.countplot(ax=ax[1], 
              data=df_cat[df_cat["OriginStateName"].isin(origin_states_filtered.index)],
              y="OriginStateName", 
              order=origin_states_filtered.index,
              hue="OriginStateName")

# Destination State
dest_state_counts = df_cat["DestStateName"].value_counts()
dest_states_filtered = dest_state_counts[dest_state_counts >= 50000]
sns.countplot(ax=ax[2], 
              data=df_cat[df_cat["DestStateName"].isin(dest_states_filtered.index)],
              y="DestStateName", 
              order=dest_states_filtered.index,
              hue="DestStateName")

# Origin City
origin_city_counts = df_cat["OriginCityName"].value_counts()
origin_cities_filtered = origin_city_counts[origin_city_counts >= 50000]
sns.countplot(ax=ax[3], 
              data=df_cat[df_cat["OriginCityName"].isin(origin_cities_filtered.index)],
              y="OriginCityName", 
              order=origin_cities_filtered.index,
              hue="OriginCityName")

# Destination City
dest_city_counts = df_cat["DestCityName"].value_counts()
dest_cities_filtered = dest_city_counts[dest_city_counts >= 50000]
sns.countplot(ax=ax[4], 
              data=df_cat[df_cat["DestCityName"].isin(dest_cities_filtered.index)],
              y="DestCityName", 
              order=dest_cities_filtered.index,
              hue="DestCityName")

# Títulos
ax[0].set_title("Airline Travel Numbers (≥50,000)", fontsize=14, fontweight='bold')
ax[1].set_title("Travel numbers from this state (≥50,000)", fontsize=14, fontweight='bold')
ax[2].set_title("Travel numbers to this state (≥50,000)", fontsize=14, fontweight='bold')
ax[3].set_title("Travel numbers from this city (≥50,000)", fontsize=14, fontweight='bold')
ax[4].set_title("Travel numbers to this city (≥50,000)", fontsize=14, fontweight='bold')

plt.tight_layout()
fig.subplots_adjust(hspace=0.25)
plt.show()'''

'fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(10,20))\nsns.set_palette("bright")\n\n# Airline\nairline_counts = df_cat["Airline"].value_counts()\nairlines_filtered = airline_counts[airline_counts >= 50000]\nsns.countplot(ax=ax[0], \n              data=df_cat[df_cat["Airline"].isin(airlines_filtered.index)],\n              y="Airline", \n              order=airlines_filtered.index,\n              hue="Airline")\n\n# Origin State\norigin_state_counts = df_cat["OriginStateName"].value_counts()\norigin_states_filtered = origin_state_counts[origin_state_counts >= 50000]\nsns.countplot(ax=ax[1], \n              data=df_cat[df_cat["OriginStateName"].isin(origin_states_filtered.index)],\n              y="OriginStateName", \n              order=origin_states_filtered.index,\n              hue="OriginStateName")\n\n# Destination State\ndest_state_counts = df_cat["DestStateName"].value_counts()\ndest_states_filtered = dest_state_counts[dest_state_counts >= 50000]\nsns.countplot(ax=ax[2], \n 

## 4.2 Numerical Variable Analysis

In [88]:
df_num.head()

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance
0,3,3,1,3,0.0,714.0,724,262.0
1,3,3,1,3,0.0,917.0,922,262.0
2,3,3,1,3,0.0,1321.0,1330,843.0
3,3,3,1,3,0.0,1636.0,1645,868.0
4,3,3,1,3,0.0,1838.0,1844,425.0


In [89]:
'''fig, ax = plt.subplots(8, 2, figsize=(10,12), gridspec_kw={"height_ratios":[6,1,6,1,6,1,6,1]})
sns.reset_defaults()

#Row_1
sns.histplot(ax = ax[0,0], data=df_num, x="DayOfWeek")
sns.boxplot(ax = ax[1,0], data=df_num, x="DayOfWeek")
sns.histplot(ax = ax[0,1], data=df_num, x="Month").set(ylabel=None)
sns.boxplot(ax = ax[1,1], data=df_num, x="Month")

#Row_2
sns.histplot(ax = ax[2,0], data=df_num, x="Quarter")
sns.boxplot(ax = ax[3,0], data=df_num, x="Quarter")
sns.histplot(ax = ax[2,1], data=df_num, x="DayofMonth").set(ylabel=None)
sns.boxplot(ax = ax[3,1], data=df_num, x="DayofMonth")

#Row_3
sns.histplot(ax = ax[4,0], data=df_num[(df_num["DepDelayMinutes"] >= 1) & (df_num["DepDelayMinutes"] <= 60)], x="DepDelayMinutes")
sns.boxplot(ax = ax[5,0], data=df_num, x="DepDelayMinutes")
sns.histplot(ax = ax[4,1], data=df_num, x="DepTime").set(ylabel=None)
sns.boxplot(ax = ax[5,1], data=df_num, x="DepTime")


#Row_4
sns.histplot(ax = ax[6,0], data=df_num, x="CRSDepTime")
sns.boxplot(ax = ax[7,0], data=df_num, x="CRSDepTime")
sns.histplot(ax = ax[6,1], data=df_num[df["Distance"] <= 2000], x="Distance")
sns.boxplot(ax = ax[7,1], data=df_num, x="Distance")

plt.tight_layout()
fig.subplots_adjust(hspace=1)
plt.show()'''

'fig, ax = plt.subplots(8, 2, figsize=(10,12), gridspec_kw={"height_ratios":[6,1,6,1,6,1,6,1]})\nsns.reset_defaults()\n\n#Row_1\nsns.histplot(ax = ax[0,0], data=df_num, x="DayOfWeek")\nsns.boxplot(ax = ax[1,0], data=df_num, x="DayOfWeek")\nsns.histplot(ax = ax[0,1], data=df_num, x="Month").set(ylabel=None)\nsns.boxplot(ax = ax[1,1], data=df_num, x="Month")\n\n#Row_2\nsns.histplot(ax = ax[2,0], data=df_num, x="Quarter")\nsns.boxplot(ax = ax[3,0], data=df_num, x="Quarter")\nsns.histplot(ax = ax[2,1], data=df_num, x="DayofMonth").set(ylabel=None)\nsns.boxplot(ax = ax[3,1], data=df_num, x="DayofMonth")\n\n#Row_3\nsns.histplot(ax = ax[4,0], data=df_num[(df_num["DepDelayMinutes"] >= 1) & (df_num["DepDelayMinutes"] <= 60)], x="DepDelayMinutes")\nsns.boxplot(ax = ax[5,0], data=df_num, x="DepDelayMinutes")\nsns.histplot(ax = ax[4,1], data=df_num, x="DepTime").set(ylabel=None)\nsns.boxplot(ax = ax[5,1], data=df_num, x="DepTime")\n\n\n#Row_4\nsns.histplot(ax = ax[6,0], data=df_num, x="CRSDepTime"

## 4.3 Analysis Categorical - Categorical

In [90]:
'''fig, axis = plt.subplots(3, 1, figsize = (10, 20))

sns.barplot(ax= axis[0], data=df, y='Airline', x='DepDelayMinutes')
sns.barplot(ax= axis[1], data=df, y='OriginStateName', x='DepDelayMinutes')
sns.barplot(ax= axis[2], data=df, y='DestStateName', x='DepDelayMinutes')

axis[0].set_title("Promedio de Delays por Aerolínea", fontsize=14, fontweight='bold')
axis[1].set_title("Promedio de Delays por lugar de origen", fontsize=14, fontweight='bold')
axis[2].set_title("Promedio de Delays por lugar de llegada", fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()'''

'fig, axis = plt.subplots(3, 1, figsize = (10, 20))\n\nsns.barplot(ax= axis[0], data=df, y=\'Airline\', x=\'DepDelayMinutes\')\nsns.barplot(ax= axis[1], data=df, y=\'OriginStateName\', x=\'DepDelayMinutes\')\nsns.barplot(ax= axis[2], data=df, y=\'DestStateName\', x=\'DepDelayMinutes\')\n\naxis[0].set_title("Promedio de Delays por Aerolínea", fontsize=14, fontweight=\'bold\')\naxis[1].set_title("Promedio de Delays por lugar de origen", fontsize=14, fontweight=\'bold\')\naxis[2].set_title("Promedio de Delays por lugar de llegada", fontsize=14, fontweight=\'bold\')\n\nplt.tight_layout()\nplt.show()'

## 4.4 Analysis Numerical - Numerical


In [91]:
df_num.columns

Index(['DayOfWeek', 'Month', 'Quarter', 'DayofMonth', 'DepDelayMinutes',
       'DepTime', 'CRSDepTime', 'Distance'],
      dtype='object')

In [92]:
'''# Tomar muestra y pre-calcular correlaciones
df_sample = df_num.sample(frac=0.5, random_state=42)
corr_matrix = df_num[['DayOfWeek', 'Month', 'Quarter', 'DayofMonth', 
                      'DepDelayMinutes', 'DepTime', 'CRSDepTime', 'Distance']].corr()

fig, axis = plt.subplots(4, 4, figsize=(20, 20))

# DayOfWeek
sns.regplot(ax=axis[0, 0], data=df_sample, x="DayOfWeek", y="DepDelayMinutes", 
            scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DayOfWeek"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 0], cbar=False)

# Month
sns.regplot(ax=axis[0, 1], data=df_sample, x="Month", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[2], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Month"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 1])

# Quarter
sns.regplot(ax=axis[0, 2], data=df_sample, x="Quarter", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[3], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Quarter"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 2])

# DayofMonth
sns.regplot(ax=axis[0, 3], data=df_sample, x="DayofMonth", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[4], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DayofMonth"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 3])

# DepTime
sns.regplot(ax=axis[2, 0], data=df_sample, x="DepTime", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[5], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DepTime"]].T, 
            annot=True, fmt=".2f", ax=axis[3, 0])

# CRSDepTime
sns.regplot(ax=axis[2, 1], data=df_sample, x="CRSDepTime", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[6], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["CRSDepTime"]].T, 
            annot=True, fmt=".2f", ax=axis[3, 1])

# Distance
sns.regplot(ax=axis[2, 2], data=df_sample, x="Distance", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[7], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Distance"]].T, 
            annot=True, fmt=".2f", ax=axis[3, 2])

# Ocultar los subplots vacíos
axis[2, 3].set_visible(False)
axis[3, 3].set_visible(False)

# Ajustar títulos
axis[0, 0].set_title("Day of Week vs Delay", fontsize=12)
axis[0, 1].set_title("Month vs Delay", fontsize=12)
axis[0, 2].set_title("Quarter vs Delay", fontsize=12)
axis[0, 3].set_title("Day of Month vs Delay", fontsize=12)
axis[2, 0].set_title("Departure Time vs Delay", fontsize=12)
axis[2, 1].set_title("Scheduled Departure Time vs Delay", fontsize=12)
axis[2, 2].set_title("Distance vs Delay", fontsize=12)

# Ajustar el diseño
plt.tight_layout()
plt.show()'''

'# Tomar muestra y pre-calcular correlaciones\ndf_sample = df_num.sample(frac=0.5, random_state=42)\ncorr_matrix = df_num[[\'DayOfWeek\', \'Month\', \'Quarter\', \'DayofMonth\', \n                      \'DepDelayMinutes\', \'DepTime\', \'CRSDepTime\', \'Distance\']].corr()\n\nfig, axis = plt.subplots(4, 4, figsize=(20, 20))\n\n# DayOfWeek\nsns.regplot(ax=axis[0, 0], data=df_sample, x="DayOfWeek", y="DepDelayMinutes", \n            scatter_kws={\'alpha\':0.3}, n_boot=100)\nsns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DayOfWeek"]].T, \n            annot=True, fmt=".2f", ax=axis[1, 0], cbar=False)\n\n# Month\nsns.regplot(ax=axis[0, 1], data=df_sample, x="Month", y="DepDelayMinutes", \n            color=sns.color_palette("colorblind")[2], scatter_kws={\'alpha\':0.3}, n_boot=100)\nsns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Month"]].T, \n            annot=True, fmt=".2f", ax=axis[1, 1])\n\n# Quarter\nsns.regplot(ax=axis[0, 2], data=df_sample, x="Quarter", y="DepDelayMinutes", \

---

# 5. Multivariate Analysis

## 5.1 Encoding categorical variables


In [93]:
# Creating encoders for categorical features and saving them as JSON files. All files prefixed with 'enc'
# contain the encoding dictionaries for each categorical feature.
df_enc = df.copy()

for column in df_cat.columns:
    unique_values = list(df_cat[column].unique())
    globals()[f"{column}_enc"] = dict(zip(unique_values, range(len(unique_values))))

    json.dump(globals()[f"{column}_enc"], open(f'../data/interim/enc_{column}.json', 'w'))


In [94]:
# Replacing the values in our categorical features to our encoded values (numerical)
for column in df_cat.columns:
    df_enc[column] = df_enc[column].map(json.load(open(f'../data/interim/enc_{column}.json')))

In [95]:
df_enc.head(3)

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName
0,3,3,1,3,0.0,714.0,724,262.0,0,0,0,0,0
1,3,3,1,3,0.0,917.0,922,262.0,0,1,1,1,1
2,3,3,1,3,0.0,1321.0,1330,843.0,0,2,2,2,2


In [96]:
del df

gc.collect()

60

## 5.2 Numerical - Categorical analysis (Correlational Analysis)

In [97]:
'''sns.pairplot(data=df_enc)
plt.show()'''

'sns.pairplot(data=df_enc)\nplt.show()'

In [98]:
'''plt.figure(figsize=(20,16))
sns.heatmap(df_enc.corr().round(2), 
            annot=True,
            mask=np.triu(df_enc.corr()),
            cmap='RdBu',
            center=0,
            square=True,
            fmt='.2f',
            annot_kws={'size': 8},  
            cbar_kws={'shrink': .8}
            )
plt.show()'''

"plt.figure(figsize=(20,16))\nsns.heatmap(df_enc.corr().round(2), \n            annot=True,\n            mask=np.triu(df_enc.corr()),\n            cmap='RdBu',\n            center=0,\n            square=True,\n            fmt='.2f',\n            annot_kws={'size': 8},  \n            cbar_kws={'shrink': .8}\n            )\nplt.show()"

---

# 6. Feature Engineering

## 6.1 Creating new features 

### - Variable de franja horaria
Esto lo hacemos debido a que usualmente los vuelos son retrasados por franjas horarias. Y si se percatan no agregaremos los minutos exactos porque esto puede hacer que los datos esten muy granulados y podria generar un Overfitting del modelo

In [99]:
def convert_military_to_quarter_hour(time):
    if pd.isna(time):
        return np.nan
    
    time_str = str(int(time)).zfill(4)
    hour = int(time_str[:2])
    minutes = int(time_str[2:])
    
    # Convertir minutos a cuartos de hora
    quarter = (minutes // 15) * 0.25
    
    # Combinar hora con cuarto de hora
    return hour + quarter
    # Ejemplos:
    # 14:00 -> 14.0
    # 14:15 -> 14.25
    # 14:30 -> 14.5
    # 14:45 -> 14.75

# Aplicar la función
df_enc['HourBlock'] = df_enc['DepTime'].apply(convert_military_to_quarter_hour)

In [100]:
df_enc["HourBlock"].dtype

dtype('float64')

In [101]:
df_enc.head()

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName,HourBlock
0,3,3,1,3,0.0,714.0,724,262.0,0,0,0,0,0,7.00
1,3,3,1,3,0.0,917.0,922,262.0,0,1,1,1,1,9.25
2,3,3,1,3,0.0,1321.0,1330,843.0,0,2,2,2,2,13.25
3,3,3,1,3,0.0,1636.0,1645,868.0,0,3,3,3,3,16.50
4,3,3,1,3,0.0,1838.0,1844,425.0,0,1,4,1,4,18.50


### Convertir mis float64 y int64 a float32 e int32

## 6.2 Outlier Analysis

In [102]:
continuos_f = list(df_enc.select_dtypes('float64'))

In [103]:
'''continuos_f = list(df_enc.select_dtypes('float64'))

fig, ax = plt.subplots(5,3, figsize=(10, 20))

row, col = 0, 0
for each in df_enc.columns:
    sns.boxplot(ax = ax[row,col], data = df_enc, x=each).tick_params(labelsize=2)
    col += 1
    if col == 3: 
        row += 1 
        col = 0

plt.tight_layout()
plt.show()'''

"continuos_f = list(df_enc.select_dtypes('float64'))\n\nfig, ax = plt.subplots(5,3, figsize=(10, 20))\n\nrow, col = 0, 0\nfor each in df_enc.columns:\n    sns.boxplot(ax = ax[row,col], data = df_enc, x=each).tick_params(labelsize=2)\n    col += 1\n    if col == 3: \n        row += 1 \n        col = 0\n\nplt.tight_layout()\nplt.show()"

In [104]:
def remove_datetime_columns(df):
    return df.select_dtypes(exclude=['datetime64[ns]'])

In [105]:
df_enc_no = df_enc.copy()

In [106]:
def remove_outliers(x, feature_name, allow_neg=True):
    q1, q3 = x.quantile([0.25, 0.75])
    iqr = q3 - q1
    upper_lim = q3 + (iqr*1.5)
    lower_lim = q1 - (iqr*1.5) if allow_neg else max(0, q1 - (iqr * 1.5))

    x = x.apply(lambda x: upper_lim if (x > upper_lim) else (lower_lim if (x < lower_lim) else x))

    filename = f'../data/interim/outliers_lims_{feature_name}.json'
    json.dump({'upper_lim': upper_lim, 'lower_lim': lower_lim}, open(filename, 'w'))

    return x

In [107]:
f_outliers_with_neg = []
f_outliers_no_neg = []

for each in continuos_f:
    if df_enc[each].min() < 0:
        f_outliers_with_neg.append(each)
    f_outliers_no_neg.append(each)

for feature in f_outliers_with_neg:
    df_enc_no[feature] = remove_outliers(df_enc_no[feature], feature)

for feature in f_outliers_no_neg:
    try:
        df_enc_no[feature] = remove_outliers(df_enc_no[feature], feature, allow_neg=False)
    except:
        continue

In [108]:
df_enc.isna().sum()[df_enc.isna().sum()>0]

Series([], dtype: int64)

In [109]:
remove_datetime_columns(df_enc)
remove_datetime_columns(df_enc_no)

,DayOfWeek,Month,Quarter,DayofMonth,DepDelayMinutes,DepTime,CRSDepTime,Distance,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName,HourBlock
0,3,3,1,3,0.0,714.0,724,262.0,0,0,0,0,0,7.00
1,3,3,1,3,0.0,917.0,922,262.0,0,1,1,1,1,9.25
2,3,3,1,3,0.0,1321.0,1330,843.0,0,2,2,2,2,13.25
3,3,3,1,3,0.0,1636.0,1645,868.0,0,3,3,3,3,16.50
4,3,3,1,3,0.0,1838.0,1844,425.0,0,1,4,1,4,18.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10390184,4,3,1,31,17.5,2014.0,1949,1167.0,8,17,32,160,83,20.00
10390185,4,3,1,17,17.5,1817.0,1733,529.0,8,20,32,101,83,18.25
10390186,2,3,1,8,17.5,2318.0,1700,723.0,8,23,2,159,2,23.25
10390187,5,3,1,25,17.5,2322.0,2129,319.0,8,35,24,103,68,23.25


In [110]:
def split(target, df, test_size=0.2, random_state=42):
    X = df.drop(columns=target)
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [111]:
# Split original dataframe 
X_train_with_outliers, X_test_with_outliers, y_train, y_test = split('DepDelayMinutes', df_enc)
X_train_without_outliers, X_test_without_outliers, _, _ = split('DepDelayMinutes', df_enc_no)

X_train_with_outliers.to_csv('../data/processed/X_train_with_outliers.csv', index=False)
X_test_with_outliers.to_csv('../data/processed/X_test_with_outliers.csv', index=False)
X_train_without_outliers.to_csv('../data/processed/X_train_without_outliers.csv', index=False)
X_test_without_outliers.to_csv('../data/processed/X_test_without_outliers.csv', index=False)

y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [112]:
X_test_with_outliers.head()

,DayOfWeek,Month,Quarter,DayofMonth,DepTime,CRSDepTime,Distance,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName,HourBlock
8688932,1,6,2,27,559.0,600,582.0,0,18,5,163,33,5.75
3495940,7,8,3,1,1815.0,1823,515.0,10,12,39,133,136,18.25
8570587,7,5,2,8,701.0,655,214.0,6,15,39,27,136,7.00
6385148,6,4,2,16,1657.0,1705,148.0,6,3,8,95,27,16.75
9803165,6,3,1,26,2007.0,2010,3417.0,10,44,39,231,136,20.00


In [113]:
def kselection(X_train, X_test, y_train, k, reference: str):
    selection_model = SelectKBest(f_classif, k=k)
    selection_model.fit(X_train, y_train)
    cols = selection_model.get_support()

    X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=X_train.columns.values[cols]) 
    X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=X_test.columns.values[cols])

    dump(selection_model, open(f'../data/processed/selection_model_{reference}.sav', 'wb'))

    return X_train_sel, X_test_sel

In [114]:
X_train_with_outliers_sel, X_test_with_outliers_sel = kselection(X_train_with_outliers, X_test_with_outliers, y_train, 'all', 'with_outliers')
X_train_without_outliers_sel, X_test_without_outliers_sel = kselection(X_train_without_outliers, X_test_without_outliers, y_train, 'all', 'without_outliers')

In [115]:
dfs_train = {
    'X_train_with_outliers_sel': X_train_with_outliers_sel,
    'X_train_without_outliers_sel': X_train_without_outliers_sel
}

dfs_test = {
    'X_test_with_outliers_sel': X_test_with_outliers_sel,
    'X_test_without_outliers_sel': X_test_without_outliers_sel   
}

for name, df in dfs_train.items():
    df.to_csv(f"../data/processed/{name}.csv", index=False)

for name, df in dfs_test.items(): 
    df.to_csv(f'../data/processed/{name}.csv', index=False)

In [116]:
train, test = [], []

for name, df in dfs_train.items():
    train.append(df)
for name, df in dfs_test.items():
    test.append(df)

In [117]:
X_train_with_outliers.columns

Index(['DayOfWeek', 'Month', 'Quarter', 'DayofMonth', 'DepTime', 'CRSDepTime',
       'Distance', 'Airline', 'OriginStateName', 'DestStateName',
       'OriginCityName', 'DestCityName', 'HourBlock'],
      dtype='object')

In [118]:
X_train_with_outliers_sel.head()

,DayOfWeek,Month,Quarter,DayofMonth,DepTime,CRSDepTime,Distance,Airline,OriginStateName,DestStateName,OriginCityName,DestCityName,HourBlock
0,1.0,7.0,3.0,4.0,2137.0,2148.0,1517.0,10.0,24.0,4.0,64.0,70.0,21.50
1,1.0,5.0,2.0,30.0,833.0,840.0,1325.0,8.0,27.0,8.0,102.0,27.0,8.50
2,5.0,6.0,2.0,24.0,1735.0,1740.0,270.0,10.0,37.0,16.0,130.0,142.0,17.50
3,3.0,6.0,2.0,16.0,1826.0,1700.0,451.0,0.0,1.0,4.0,57.0,12.0,18.25
4,2.0,6.0,2.0,29.0,1812.0,1805.0,323.0,6.0,3.0,8.0,62.0,205.0,18.00


In [119]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def format_delay_time(minutes):
    """
    Convierte minutos en un formato legible de horas y minutos
    """
    hours = int(minutes // 60)
    mins = int(minutes % 60)
    if hours > 0:
        return f"{hours}h {mins}min"
    return f"{mins}min"

results = []

# Parámetros base para XGBoost
xgb_params = {
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 100,
    'objective': 'reg:squarederror',
    'random_state': 42
}

for index in range(len(train)):
    # Crear y entrenar el modelo XGBoost
    model = XGBRegressor(**xgb_params)
    train_df = train[index]
    model.fit(
        train_df, 
        y_train,
        eval_set=[(train_df, y_train)],
        verbose=False
    )
    
    # Realizar predicciones
    y_test_pred = model.predict(test[index])
    
    # Calcular predicciones medias y máximas
    mean_delay = np.mean(y_test_pred)
    max_delay = np.max(y_test_pred)
    
    # Obtener importancia de características
    feature_importance = model.feature_importances_
    
    results.append({
        'index': index,
        'df_train': list(dfs_train.keys())[index],
        'Feature_importance': feature_importance,
        'MAE': mean_absolute_error(y_test, y_test_pred).round(2),
        'RMSE': round(root_mean_squared_error(y_test, y_test_pred), 2),
        'R2_score': round(r2_score(y_test, y_test_pred), 5),
        'Predicted_Mean_Delay': format_delay_time(mean_delay),
        'Predicted_Max_Delay': format_delay_time(max_delay),
        'model': model
    })

# Ordenar resultados por RMSE
results = sorted(results, key=lambda x: x['RMSE'])
best_ind = results[0]['index']
best_df_train = results[0]['df_train']
best_model = results[0]['model']

# Mostrar resultados sin el modelo
results_display = [{k: v for k, v in r.items() if k != 'model'} for r in results]

# Mostrar estadísticas de retraso para el mejor modelo
print("\nEstadísticas de retraso del mejor modelo:")
best_predictions = best_model.predict(test[best_ind])
print(f"Retraso promedio esperado: {format_delay_time(np.mean(best_predictions))}")
print(f"Retraso máximo esperado: {format_delay_time(np.max(best_predictions))}")
print(f"Retraso mínimo esperado: {format_delay_time(np.min(best_predictions))}")

# Para hacer una predicción específica con el mejor modelo:
def predict_delay(model, new_data):
    prediction = model.predict([new_data])[0]
    return format_delay_time(prediction)

# Ejemplo de uso:
# delay = predict_delay(best_model, test[best_ind].iloc[0])
# print(f"Retraso predicho para este vuelo: {delay}")

results_display


Estadísticas de retraso del mejor modelo:
Retraso promedio esperado: 14min
Retraso máximo esperado: 25h 27min
Retraso mínimo esperado: 56min


[{'index': 0,
  'df_train': 'X_train_with_outliers_sel',
  'Feature_importance': array([0.00246432, 0.01228601, 0.        , 0.00142395, 0.19225317,
         0.53265804, 0.01975912, 0.00758026, 0.00726532, 0.01103152,
         0.00612486, 0.00268298, 0.20447038], dtype=float32),
  'MAE': 3.54,
  'RMSE': 12.84,
  'R2_score': 0.93299,
  'Predicted_Mean_Delay': '14min',
  'Predicted_Max_Delay': '25h 27min'},
 {'index': 1,
  'df_train': 'X_train_without_outliers_sel',
  'Feature_importance': array([0.00248439, 0.01100739, 0.        , 0.00207453, 0.19181818,
         0.52657604, 0.02282179, 0.00842188, 0.00766283, 0.00737707,
         0.00536882, 0.01694752, 0.19743957], dtype=float32),
  'MAE': 3.61,
  'RMSE': 12.91,
  'R2_score': 0.93224,
  'Predicted_Mean_Delay': '14min',
  'Predicted_Max_Delay': '24h 43min'}]

In [120]:
results[results[0].get("index")].get('MAE')

3.54

In [121]:
print(f'Our best dataframe for our model is {best_df_train}, with a MAE of {results[results[0].get("index")].get("MAE")}, \
a RMSE of {results[results[0].get("index")].get("RMSE")} and a R2_score of {results[results[0].get("index")].get("R2_score")}')

Our best dataframe for our model is X_train_with_outliers_sel, with a MAE of 3.54, a RMSE of 12.84 and a R2_score of 0.93299


In [122]:
# Verificar que el modelo está prediciendo valores diferentes en el conjunto de prueba original
y_test_pred = best_model.predict(test[best_ind])
print("Muestra de predicciones en el conjunto de prueba:")
print(y_test_pred[:10])  # Primeras 10 predicciones
print(f"\nRango de predicciones:")
print(f"Mínimo: {min(y_test_pred)}")
print(f"Máximo: {max(y_test_pred)}")
print(f"Media: {np.mean(y_test_pred)}")

Muestra de predicciones en el conjunto de prueba:
[1.7848033 2.7839298 3.9961362 2.863127  2.4830184 1.7073576 3.1245356
 2.4707952 2.0766022 1.3044893]

Rango de predicciones:
Mínimo: -3.222576141357422
Máximo: 1527.4962158203125
Media: 14.079286575317383


In [123]:
# Crear casos con valores extremos
casos_extremos = pd.DataFrame({
    'DayOfWeek':       [1.0, 7.0],    
    'Month':           [1.0, 12.0],    
    'Quarter':         [1.0, 4.0],    
    'DayofMonth':      [1.0, 31.0],    
    'DepTime':         [0.0, 2359.0], 
    'CRSDepTime':      [0.0, 2359.0],
    'Distance':        [100.0, 3000.0],
    'Airline':         [min(X_train_with_outliers_sel['Airline']), max(X_train_with_outliers_sel['Airline'])],
    'OriginStateName': [min(X_train_with_outliers_sel['OriginStateName']), max(X_train_with_outliers_sel['OriginStateName'])],
    'DestStateName':   [min(X_train_with_outliers_sel['DestStateName']), max(X_train_with_outliers_sel['DestStateName'])],
    'OriginCityName':  [min(X_train_with_outliers_sel['OriginCityName']), max(X_train_with_outliers_sel['OriginCityName'])],
    'DestCityName':    [min(X_train_with_outliers_sel['DestCityName']), max(X_train_with_outliers_sel['DestCityName'])],
    'HourBlock':       [0.0, 23.0]
})

predicciones_extremas = best_model.predict(casos_extremos)
print("\nPredicciones con valores extremos:")
for i, pred in enumerate(predicciones_extremas):
    print(f"Caso extremo {i+1}: {format_delay_time(pred)}")


Predicciones con valores extremos:
Caso extremo 1: 3min
Caso extremo 2: 31min


In [124]:
casos_prueba = pd.DataFrame({
    # 1: Peor escenario - Vuelo en temporada alta, viernes por la tarde
    # 2: Mejor escenario - Vuelo temprano, día entre semana
    # 3: Caso típico - Vuelo de media distancia en horario normal
    # 4: Caso de alto riesgo - Último vuelo del día
    'DayOfWeek':       [5.0,   2.0,    3.0,    7.0],    
    'Month':           [12.0,  4.0,    7.0,    7.0],    
    'Quarter':         [4.0,   2.0,    3.0,    3.0],    
    'DayofMonth':      [23.0,  12.0,   15.0,   20.0],    
    'DepTime':         [1700.0, 600.0, 1400.0, 2300.0],
    'CRSDepTime':      [1700.0, 600.0, 1400.0, 2300.0],
    'Distance':        [2500.0, 500.0, 1000.0, 800.0],
    'Airline':         [11.0,  16.0,   7.0,    20.0],    
    'OriginStateName': [6.0,   12.0,   32.0,   17.0],   
    'DestStateName':   [36.0,  35.0,   23.0,   23.0],
    'OriginCityName':  [116.0, 21.0,   140.0,  87.0],
    'DestCityName':    [104.0, 135.0,  41.0,   44.0],
    'HourBlock':       [17.0,  6.0,    14.0,   23.0]    
})

predicciones = best_model.predict(casos_prueba)

print("Análisis detallado de predicciones:")

print("\nCaso 1 - Peor escenario (Viernes, temporada alta):")
print(f"Fecha: Viernes 23 de Diciembre")
print(f"Hora: 17:00 (Rush hour)")
print(f"Distancia: 2500 millas (Vuelo largo)")
print(f"Retraso predicho: {format_delay_time(predicciones[0])}")

print("\nCaso 2 - Mejor escenario (Martes temprano):")
print(f"Fecha: Martes 12 de Abril")
print(f"Hora: 06:00")
print(f"Distancia: 500 millas (Vuelo corto)")
print(f"Retraso predicho: {format_delay_time(predicciones[1])}")

print("\nCaso 3 - Caso típico (Mitad de semana):")
print(f"Fecha: Miércoles 15 de Julio")
print(f"Hora: 14:00")
print(f"Distancia: 1000 millas")
print(f"Retraso predicho: {format_delay_time(predicciones[2])}")

print("\nCaso 4 - Último vuelo del día:")
print(f"Fecha: Domingo 20 de Julio")
print(f"Hora: 23:00")
print(f"Distancia: 800 millas")
print(f"Retraso predicho: {format_delay_time(predicciones[3])}")

Análisis detallado de predicciones:

Caso 1 - Peor escenario (Viernes, temporada alta):
Fecha: Viernes 23 de Diciembre
Hora: 17:00 (Rush hour)
Distancia: 2500 millas (Vuelo largo)
Retraso predicho: 2min

Caso 2 - Mejor escenario (Martes temprano):
Fecha: Martes 12 de Abril
Hora: 06:00
Distancia: 500 millas (Vuelo corto)
Retraso predicho: 0min

Caso 3 - Caso típico (Mitad de semana):
Fecha: Miércoles 15 de Julio
Hora: 14:00
Distancia: 1000 millas
Retraso predicho: 3min

Caso 4 - Último vuelo del día:
Fecha: Domingo 20 de Julio
Hora: 23:00
Distancia: 800 millas
Retraso predicho: 5min
